In [1]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import pprint
import os
from bs4 import BeautifulSoup
import requests
import json
import re

In [2]:
# Get a client (the "core" client provides access to projects, teams, etc)
def get_clients_projects():

    # Fill in with your personal access token and org URL
    personal_access_token = os.getenv("gitpat")
    organization_url = 'https://dev.azure.com/supportedcustomers'

    # Create a connection to the org
    credentials = BasicAuthentication('', personal_access_token)
    connection = Connection(base_url=organization_url, creds=credentials)    
    
    core_client = connection.clients.get_core_client()
    git_client = connection.clients.get_git_client()

    # Get the first page of projects
    get_projects_response = core_client.get_projects()
    return core_client, git_client, get_projects_response

In [3]:
def recurse_git_tree(repo_id, object_id=None):
    print(f'step one')
    tree = git_client.get_tree(repository_id=repo.id, sha1=object_id)

    for entry in tree.tree_entries:
        if entry.relative_path=="pipeline":
            print(f'step two')
            recurse_git_tree(repo_id=repo_id, object_id=entry.object_id)
        elif entry.git_object_type =='blob':
            print(f'step three')
            blob = git_client.get_blob_content(repository_id=repo.id, sha1=entry.object_id)
            s = json.dumps(json.loads(next(blob)))

            try:
                if s.index("\"type\": \"Copy\"")>0:               
                    print(f'before: {s}')
                    found_value = s[s.index("\"timeout\""): s.find(",", s.index("timeout"))]
                    s = s.replace(found_value,'"timeout": "5:00:00"')
                    found_value = s[s.index("\"retryIntervalInSeconds\""): s.find(",", s.index("retryIntervalInSeconds"))]
                    s = s.replace(found_value,'"retryIntervalInSeconds": "2200"')
                    found_value = s[s.index("\"retry\""): s.find(",", s.index("retry"))]
                    s = s.replace(found_value,'"retry": "1"')
                    print(f'after: {s}')
            except Exception as e:
                print(e)
                continue
        else:
            recurse_git_tree(repo_id=repo_id, object_id=entry.object_id)


### Work in progress

In [ ]:
core_client, git_client, get_projects_response = get_clients_projects()

index = 0
while get_projects_response is not None:
    print('did i get')
    for project in get_projects_response.value:
        repos = git_client.get_repositories(project.id)
        for repo in repos:
            if repo.name=="noname":
                try:
                    branches = git_client.get_branches(repository_id=repo.id)
                    for branch in branches:
                        print(branch.commit.commit_id)
                        print(f'The repository {repo.name} has the following branch {branch.name}')
                        commit = git_client.get_commit(repository_id=repo.id, commit_id=branch.commit.commit_id)

                        recurse_git_tree(repo_id=repo.id, object_id=commit.tree_id)

                except Exception as e:
                    print(e)
                    continue
            index += 1
    if get_projects_response.continuation_token is not None and get_projects_response.continuation_token != "":
        # Get the next page of projects
        get_projects_response = core_client.get_projects(continuation_token=get_projects_response.continuation_token)

    else:
        # All projects have been retrieved
        get_projects_response = None

The repository noname has the following branch adf2_test1
{
    'additional_properties': {}, 
    'git_object_type': 'tree', 
    'mode': '40000', 
    'object_id': 'aa4e7709a0d2577a07e31358f06dac9657d00963', 
    'relative_path': 'pipeline', 
    'size': 89, 
    'url': 'https://dev.azure.com/SupportedCustomers/e6091a89-c16c-440b-a71b-e4d9151542a1/_apis/git/repositories/6b058b4d-221d-4ca1-905b-5f2eda2a2462/trees/aa4e7709a0d2577a07e31358f06dac9657d00963'
}

In [5]:
core_client, git_client, get_projects_response = get_clients_projects()

index = 0
while get_projects_response is not None:
    for project in get_projects_response.value:
        repos = git_client.get_repositories(project.id)
        for repo in repos:
            try:
                branches = git_client.get_branches(repository_id=repo.id)
                for branch in branches:
                    print(branch)
                    #print(branch.commit.commit_id)
                    print(f'The repository {repo.name} has the following branch {branch.name}')
                    commit = git_client.get_commit(repository_id=repo.id, commit_id=branch.commit.commit_id)

                    print(f'Is this the oldobjectId that I am looking for? {commit.commit_id}')
                    tree = git_client.get_tree(repository_id=repo.id, sha1=commit.tree_id)

                    #refs = git_client.get_refs(repo)
                    #print(f'This is the commit: {refs.commit}')

                    for entry in tree.tree_entries:
                        if entry.relative_path == "pipeline":
                            cc = git_client.get_tree(repository_id=repo.id, sha1=entry.object_id)
                            for c in cc.tree_entries:
                                if c.git_object_type == 'blob':
                                    blob = git_client.get_blob_content(repository_id=repo.id, sha1=c.object_id)
                                    data = json.loads(next(blob))

                                    for d in data:
                                        print(f'what is the type value of data[d] {type(data[d])}')
                                        if type(data[d])=="str":
                                            print(f'this is the value of data[d] {data[d]}')


                                    s = json.dumps(data)

                                    if s.index("\"type\": \"Copy\""):               
                                        print(f'before: {s}')

                                        found_value = s[s.index("\"timeout\""): s.find(",", s.index("timeout"))]
                                        s = s.replace(found_value,'"timeout": "5:00:00"')
                                        found_value = s[s.index("\"retryIntervalInSeconds\""): s.find(",", s.index("retryIntervalInSeconds"))]
                                        s = s.replace(found_value,'"retryIntervalInSeconds": "2200"')
                                        found_value = s[s.index("\"retry\""): s.find(",", s.index("retry"))]
                                        s = s.replace(found_value,'"retry": "1"')

                                        print(f'after: {s}')

            except Exception as e:
                print(e)
                continue
        index += 1

        #project.
    if get_projects_response.continuation_token is not None and get_projects_response.continuation_token != "":
        # Get the next page of projects
        get_projects_response = core_client.get_projects(continuation_token=get_projects_response.continuation_token)

    else:
        # All projects have been retrieved
        get_projects_response = None

{'additional_properties': {}, 'ahead_count': 1, 'behind_count': 3, 'commit': <azure.devops.v5_1.git.models.GitCommitRef object at 0x7faed65597f0>, 'is_base_version': False, 'name': 'adf2_test1'}
The repository Ecolab has the following branch adf2_test1
A potentially dangerous Request.Path value was detected from the client (:).
{'additional_properties': {}, 'ahead_count': 0, 'behind_count': 0, 'commit': <azure.devops.v5_1.git.models.GitCommitRef object at 0x7faed656ba60>, 'is_base_version': True, 'name': 'main'}
The repository PBI_Pipelines has the following branch main
A potentially dangerous Request.Path value was detected from the client (:).
{'additional_properties': {}, 'ahead_count': 1, 'behind_count': 3, 'commit': <azure.devops.v5_1.git.models.GitCommitRef object at 0x7faed6526880>, 'is_base_version': False, 'name': 'adf_publish'}
The repository Data-Factory has the following branch adf_publish
A potentially dangerous Request.Path value was detected from the client (:).
{'additi

In [ ]:
#git_client.create_thread(repository_id=repo.id, comment_thread='this is something')
git_client.update_repository(repository_id=repo.id)

In [ ]:
organization_url = 'https://dev.azure.com/supportedcustomers'
pull_request_id=git_client.create_pull_request(repository_id=repo.id, git_pull_request_to_create=[organization_url,'test1'])
#git_client.get_pull_request(repository_id=repo.id, pull_request_id=pull_request_id)